# Sizing for MOSFET Diode Example 

**Copyright 2024 Harald Pretl**

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at
http://www.apache.org/licenses/LICENSE-2.0

In [1]:
# read table data
from pygmid import Lookup as lk
import numpy as np
lv_pmos = lk('../backup/gf180mcu_pmos_3v3.mat')
lv_nmos = lk('../backup/gf180mcu_nmos_3v3.mat')
# list of parameters: VGS, VDS, VSB, L, W, NFING, ID, VT, GM, GMB, GDS, CGG, CGB, CGD, CGS, CDD, CSS, STH, SFL
# if not specified, minimum L, VDS=max(vgs)/2=0.9 and VSB=0 are used 

In [2]:
# define the given values
id_spec = 20e-6
gm_id_spec = 10
L_spec = 0.13

In [3]:
# we can calculate the gm directly
gm = gm_id_spec * id_spec
print('gm =', round(gm/1e-3, 3), 'mS')

gm = 0.2 mS


In [4]:
vgs = lv_nmos.look_upVGS(GM_ID=gm_id_spec, L=L_spec, VDS=1.65, VSB=0.0)
vgs = lv_nmos.look_upVGS(GM_ID=gm_id_spec, L=L_spec, VDS=vgs, VSB=0.0)

In [5]:
# the gm_gds we look up and calculate gds from it
gm_gds = lv_nmos.lookup('GM_GDS', GM_ID=gm_id_spec, L=L_spec, VDS=vgs, VSB=0)
gds = gm / gm_gds
print('gds =', round(gds/1e-6, 2), 'uS')

gds = -4.47 uS


In [6]:
# find f_T (which is not stored directly, but we can find the gm to gate capacitance ratio)
gm_cgg = lv_nmos.lookup('GM_CGG', GM_ID=gm_id_spec, L=L_spec, VDS=vgs, VSB=0)
f_T = gm_cgg / (2*np.pi)
print('Cgg =', round(gm/gm_cgg/1e-15, 1), 'fF')
print('f_T =', round(f_T/1e9, 1), 'GHz')


Cgg = 3.6 fF
f_T = 8.8 GHz


In [7]:
# find the W of the diode transistor
id_w = lv_nmos.lookup('ID_W', GM_ID=gm_id_spec, L=L_spec, VDS=vgs, VSB=0)
w = id_spec / id_w
print('W =', round(w, 2), 'um, rounded W =', round(w*2)/2, 'um')

W = 4.77 um, rounded W = 5.0 um


In [8]:
# let's now find the other interesting values
# lookup normalized noise power spectral densisties and denormalize with the gm from above
sth = lv_nmos.lookup('STH_GM', VGS=vgs, L=L_spec, VDS=vgs, VSB=0)*gm
sfl = lv_nmos.lookup('SFL_GM', VGS=vgs, L=L_spec, VDS=vgs, VSB=0)*gm

gamma = sth/(4*1.38e-23*300*gm)

f_co = sfl/sth
print('V_GS =', round(float(vgs), 3), 'V')
print('gamma =', round(gamma, 2))
print('f_co =', round(f_co/1e6, 2), 'MHz')

V_GS = 0.677 V
gamma = 0.0
f_co = nan MHz


/tmp/ipykernel_1282/607909554.py:8: RuntimeWarning: invalid value encountered in scalar divide
  f_co = sfl/sth


In [9]:
sfl

0.0